### R06725035 陳廷易
* feature selection: chi-square, log likelihood ratios, expected mutual information
* naive bayes classification
* voting

In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from nltk import word_tokenize
from nltk.stem.porter import *
from nltk.tokenize import RegexpTokenizer
import os,sys
# import re
import pickle
import pandas as pd
import numpy as np
import tqdm
from tqdm import tqdm
import random
import math
from collections import Counter
import functools

In [2]:
with open('data/stop_words.txt') as f:
    stop_words_list = f.read().splitlines() #stop_list1
stop_list2 = pickle.load(open('data/stop_list2.pkl','rb'))
ps = PorterStemmer() # Stemming
stop_words = set(stopwords.words('english')) #Stopword
short = ['.', ',', '"', "\'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}', "'at",
         "_","`","\'\'","--","``",".,","//",":","___",'_the','-',"'em",".com",
                   '\'s','\'m','\'re','\'ll','\'d','n\'t','shan\'t',"...","\'ve",'u']
stop_words_list.extend(short)
stop_words_list.extend(stop_list2)
stop_words.update(stop_words_list)

In [3]:
def preprocess(texts):
    tokens = [i for i in word_tokenize(texts.lower()) if i not in stop_words]  # Tokenization.# Lowercasing
    token_result = ''
    token_result_ = ''
    for i,token in enumerate(tokens): #list2str
        token_result += ps.stem(token) + ' '
    token_result = ''.join([i for i in token_result if not i.isdigit()])
    token_result = [i for i in word_tokenize(token_result) if i not in stop_words]
    for i,token in enumerate(token_result):
        token_result_ += token + ' '
    return token_result_

## Feature Selection
* tf-idf
* chi-square
* likelihood
* PMI
* EMI

=> build dictionary in 500 words

In [4]:
dict_df = pd.read_csv('data/dictionary.txt',header=None,index_col=None,sep=' ')
terms = dict_df[1].tolist() #all terms

In [5]:
with open('data/training.txt','r') as f:
    train_id = f.read().splitlines()
train_dict = {}
for trainid in train_id:
    trainid = trainid.split(' ')
    trainid = list(filter(None, trainid))
    train_dict[trainid[0]] = trainid[1:]
print(train_dict) #class:doc_id
train_dict = pickle.load(open('data/train_dict.pkl','rb'))



{'1': ['11', '19', '29', '113', '115', '169', '278', '301', '316', '317', '321', '324', '325', '338', '341'], '2': ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '12', '13', '14', '15', '16'], '3': ['813', '817', '818', '819', '820', '821', '822', '824', '825', '826', '828', '829', '830', '832', '833'], '4': ['635', '680', '683', '702', '704', '705', '706', '708', '709', '719', '720', '722', '723', '724', '726'], '5': ['646', '751', '781', '794', '798', '799', '801', '812', '815', '823', '831', '839', '840', '841', '842'], '6': ['995', '998', '999', '1003', '1005', '1006', '1007', '1009', '1011', '1012', '1013', '1014', '1015', '1016', '1019'], '7': ['700', '730', '731', '732', '733', '735', '740', '744', '752', '754', '755', '756', '757', '759', '760'], '8': ['262', '296', '304', '308', '337', '397', '401', '443', '445', '450', '466', '480', '513', '533', '534'], '9': ['130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '145'], '10': 

In [6]:
in_dir = 'data/IRTM/'
train_dict_ = {}
class_token = []
class_token_dict = {}
for c,d in train_dict.items():
    for doc in d:
        f = open('data/IRTM/'+doc+'.txt')
        texts = f.read()
        f.close()
        tokens_all = preprocess(texts)
        tokens_all = tokens_all.split(' ')
        tokens_all = list(set(filter(None,tokens_all)))
        class_token.append(tokens_all)
    class_token_dict[c]=class_token
    class_token=[]
# len(class_token_dict['1'])

In [7]:
dict_df.drop(0,axis=1,inplace=True)
dict_df.columns = ['term','score']
dict_df.index = dict_df['term']
dict_df.drop('term',axis=1,inplace=True)
print(dict_df)

             score
term              
abandon         39
abc             51
abcnews.com      3
abdallah         2
abdel            3
abdomin          2
abduct           2
abdul           53
abdul-karim     11
abdullah        40
abhorr           2
abid            48
abidjan         36
abil            80
abl            319
ablaz            3
aboard          93
abobo            3
abolish          4
abort            7
abou            29
abound           4
abraham         10
abroad          52
abrog            2
abrupt           5
abruptli         4
absenc           6
absent          14
absente          7
...            ...
young          103
younger         20
youngest         6
youngster        3
youth           15
youzhni          2
yu-              2
yug            139
yugoslav       139
yugoslavia     130
yugoslavian     15
yuri             3
yuxi             2
zacharea         2
zan             47
zarko            6
zawahri          5
zeal             3
zedek            2
zein        

In [8]:
dict_df['score'] = 0
dict_df['score_chi'] = 0
dict_df['score_emi'] = 0
# c=1
for term in tqdm(terms): #each term
    scores = []
    scores_chi = []
    scores_emi = []
    c=1
    for _ in range(len(class_token_dict)): # each class
        n11=e11=m11=0
        n10=e10=m10=0
        n01=e01=m01=0
        n00=e00=m00=0
        for k,v in class_token_dict.items():
#             print(k,c)
            if k == str(c): #ontopic
                for r in v:
                    if term in r:
                        n11+=1
                    else:
                        n10+=1
#                 c+=1
            else: #off topic
                for r in v:
                    if term in r:
                        n01+=1
                    else:
                        n00+=1
#                 c+=1
        c+=1
        n11+=1e-8
        n10+=1e-8
        n01+=1e-8
        n00+=1e-8
        N = n11+n10+n01+n00
        e11 = N * (n11+n01)/N * (n11+n10)/N #chi-squre
        e10 = N * (n11+n10)/N * (n10+n00)/N
        e01 = N * (n11+n01)/N * (n01+n00)/N
        e00 = N * (n01+n00)/N * (n10+n00)/N
        score_chi = ((n11-e11)**2)/e11 + ((n10-e10)**2)/e10 + ((n01-e01)**2)/e01 + ((n00-e00)**2)/e00
        scores_chi.append(score_chi)
        
        n11 = n11 - 1e-8 + 1e-6
        n10 = n10 - 1e-8 + 1e-6
        n01 = n01 - 1e-8 + 1e-6
        n00 = n00 - 1e-8 + 1e-6
        N = n11+n10+n01+n00
        m11 = (n11/N) * math.log(((n11/N)/((n11+n01)/N * (n11+n10)/N)),2) #EMI
        m10 = n10/N * math.log((n10/N)/((n11+n10)/N * (n10+n00)/N),2)
        m01 = n01/N * math.log((n01/N)/((n11+n01)/N * (n01+n00)/N),2)
        m00 = n00/N * math.log((n00/N)/((n01+n00)/N * (n10+n00)/N),2)
        score_emi = m11 + m10 + m01 + m00
        scores_emi.append(score_emi)
        
#         print(n11,n10,n01,n00)
        n11-=1e-6
        m10-=1e-6
        n01-=1e-6
        n00-=1e-6
        N = n11+n10+n01+n00
        score = (((n11+n01)/N) ** n11) * ((1 - ((n11+n01)/N)) ** n10) * (((n11+n01)/N) ** n01) * ((1 - ((n11+n01)/N)) ** n00)
        score /= ((n11/(n11+n10)) ** n11) * ((1 - (n11/(n11+n10))) ** n10) * ((n01/(n01+n00)) ** n01) * ((1 - (n01/(n01+n00))) ** n00)
        score = -2 * math.log(score, 10) #LLR
        scores.append(score)
        
        
#         c+=1
    dict_df.loc[term,'score'] = np.mean(scores)
    dict_df.loc[term,'score_chi'] = np.mean(scores_chi)
    dict_df.loc[term,'score_emi'] = np.mean(scores_emi)
print(dict_df)    

100%|██████████| 8913/8913 [03:55<00:00, 37.79it/s]

                 score     score_chi     score_emi
term                                              
abandon       1.659552  3.846051e+00  2.913791e-03
abc           0.899990  1.555876e+00  1.580176e-03
abcnews.com   0.204845  6.544514e-01  3.596454e-04
abdallah      0.274067  5.290393e-01  4.811811e-04
abdel         0.411467  7.944005e-01  7.224236e-04
abdomin       0.274067  5.290393e-01  4.811811e-04
abduct        0.160386  3.668540e-01  2.815866e-04
abdul         0.464110  9.267195e-01  8.148566e-04
abdul-karim   1.519594  2.937724e+00  2.668045e-03
abdullah      5.383307  1.045027e+01  9.451857e-03
abhorr        0.599975  4.723560e+00  1.053402e-03
abid          0.378774  8.999763e-01  6.650296e-04
abidjan      10.299234  5.488540e+01  1.808312e-02
abil          0.976184  2.298259e+00  1.713953e-03
abl           0.970499  1.925329e+00  1.703978e-03
ablaz         0.293008  1.039403e+00  5.144410e-04
aboard        6.593954  1.392154e+01  1.157749e-02
abobo         0.812945  4.68362

In [9]:
dict_df2 = pd.read_csv('data/dictionary.txt',header=None,index_col=None,sep=' ')
dict_df2.columns = ['id','term','freq']
dict_df2['sum'] = 0.0
print(dict_df2)

        id         term  freq  sum
0        1      abandon    39  0.0
1        2          abc    51  0.0
2        3  abcnews.com     3  0.0
3        4     abdallah     2  0.0
4        5        abdel     3  0.0
5        6      abdomin     2  0.0
6        7       abduct     2  0.0
7        8        abdul    53  0.0
8        9  abdul-karim    11  0.0
9       10     abdullah    40  0.0
10      11       abhorr     2  0.0
11      12         abid    48  0.0
12      13      abidjan    36  0.0
13      14         abil    80  0.0
14      15          abl   319  0.0
15      16        ablaz     3  0.0
16      17       aboard    93  0.0
17      18        abobo     3  0.0
18      19      abolish     4  0.0
19      20        abort     7  0.0
20      21         abou    29  0.0
21      22       abound     4  0.0
22      23      abraham    10  0.0
23      24       abroad    52  0.0
24      25        abrog     2  0.0
25      26       abrupt     5  0.0
26      27     abruptli     4  0.0
27      28       abs

In [10]:
# import os
tf_list = next(os.walk('data/tf-idf/'))[2]
# df_list = [dict_df2]
for tf in tf_list:
#     print(tf)
    df2 = pd.read_csv('data/tf-idf/'+tf,header=None,index_col=None,sep=' ',skiprows=[0])
    df2.columns = ['id','tfidf']
    df3 = pd.merge(dict_df2,df2,on='id',how='outer')
    df3.fillna(0,inplace=True)
    dict_df2['sum']+=df3['tfidf']
dict_df2['avg_tfidf'] = dict_df2['sum']/dict_df2['freq']
dict_df2 = dict_df2.drop(['freq','sum'],axis=1)
print(dict_df2)
#     break
#     df_list.append(df2)
# df3 = pd.concat(df_list).groupby(level=0).sum()
# df3 = pd.concat([df3,df2]).groupby(level=0).sum()
# df3 = pd.merge(dict_df2,df2,on='id',how='outer')
# df3 = pd.merge(df3,df2,on='id',how='outer')?
# df3[df3.id == 2]

        id         term  avg_tfidf
0        1      abandon   0.005374
1        2          abc   0.015857
2        3  abcnews.com   0.033696
3        4     abdallah   0.006597
4        5        abdel   0.003924
5        6      abdomin   0.010949
6        7       abduct   0.016260
7        8        abdul   0.000851
8        9  abdul-karim   0.005135
9       10     abdullah   0.006808
10      11       abhorr   0.008349
11      12         abid   0.000591
12      13      abidjan   0.017659
13      14         abil   0.001223
14      15          abl   0.000976
15      16        ablaz   0.006857
16      17       aboard   0.006911
17      18        abobo   0.005802
18      19      abolish   0.006744
19      20        abort   0.012137
20      21         abou   0.000133
21      22       abound   0.007865
22      23      abraham   0.007256
23      24       abroad   0.006570
24      25        abrog   0.003322
25      26       abrupt   0.001282
26      27     abruptli   0.026259
27      28       abs

In [11]:
dict_df['term'] = dict_df.index
dict_df3 = pd.merge(dict_df,dict_df2,on='term',how='outer')
print(dict_df3)

          score     score_chi     score_emi         term    id  avg_tfidf
0      1.659552  3.846051e+00  2.913791e-03      abandon     1   0.005374
1      0.899990  1.555876e+00  1.580176e-03          abc     2   0.015857
2      0.204845  6.544514e-01  3.596454e-04  abcnews.com     3   0.033696
3      0.274067  5.290393e-01  4.811811e-04     abdallah     4   0.006597
4      0.411467  7.944005e-01  7.224236e-04        abdel     5   0.003924
5      0.274067  5.290393e-01  4.811811e-04      abdomin     6   0.010949
6      0.160386  3.668540e-01  2.815866e-04       abduct     7   0.016260
7      0.464110  9.267195e-01  8.148566e-04        abdul     8   0.000851
8      1.519594  2.937724e+00  2.668045e-03  abdul-karim     9   0.005135
9      5.383307  1.045027e+01  9.451857e-03     abdullah    10   0.006808
10     0.599975  4.723560e+00  1.053402e-03       abhorr    11   0.008349
11     0.378774  8.999763e-01  6.650296e-04         abid    12   0.000591
12    10.299234  5.488540e+01  1.80831

/home/leoqaz12/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2961: FutureWarning: 'term' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  exec(code_obj, self.user_global_ns, self.user_ns)


In [12]:
cols = list(dict_df3)
cols[4], cols[3], cols[5], cols[1], cols[0], cols[2] = cols[0], cols[1] , cols[2] , cols[3], cols[4], cols[5]
dict_df3 = dict_df3.ix[:,cols]
print(dict_df3)

        id         term  avg_tfidf     score_chi      score     score_emi
0        1      abandon   0.005374  3.846051e+00   1.659552  2.913791e-03
1        2          abc   0.015857  1.555876e+00   0.899990  1.580176e-03
2        3  abcnews.com   0.033696  6.544514e-01   0.204845  3.596454e-04
3        4     abdallah   0.006597  5.290393e-01   0.274067  4.811811e-04
4        5        abdel   0.003924  7.944005e-01   0.411467  7.224236e-04
5        6      abdomin   0.010949  5.290393e-01   0.274067  4.811811e-04
6        7       abduct   0.016260  3.668540e-01   0.160386  2.815866e-04
7        8        abdul   0.000851  9.267195e-01   0.464110  8.148566e-04
8        9  abdul-karim   0.005135  2.937724e+00   1.519594  2.668045e-03
9       10     abdullah   0.006808  1.045027e+01   5.383307  9.451857e-03
10      11       abhorr   0.008349  4.723560e+00   0.599975  1.053402e-03
11      12         abid   0.000591  8.999763e-01   0.378774  6.650296e-04
12      13      abidjan   0.017659  5.

/home/leoqaz12/.local/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  app.launch_new_instance()


In [13]:
dict_df3.columns = ['id','term','avg_tfidf','score_chi','score_llr','score_emi']
# dict_df3.to_csv('output/feature_selection_df_rev.csv')

### select top 500
* 取各col的mean+1.45*std
* 再去做投票，超過兩票的流下來看剩下哪幾個

In [14]:
dict_df3 = pd.read_csv('output/feature_selection_df_rev.csv',index_col=None)
threshold_tfidf = np.mean(dict_df3['avg_tfidf'])+2.5*np.std(dict_df3['avg_tfidf']) #1.45=>502 數字大嚴格
threshold_chi = np.mean(dict_df3['score_chi'])+2.5*np.std(dict_df3['score_chi']) #1=>350
threshold_llr = np.mean(dict_df3['score_llr'])+2.5*np.std(dict_df3['score_llr']) #1.75=>543
threshold_emi = np.mean(dict_df3['score_emi'])+2.5*np.std(dict_df3['score_emi']) #1.75=>543

print('avg_tfidf',threshold_tfidf)
# dict_df3[dict_df3.score_llr>0.1]

avg_tfidf 0.024888109121963365


In [15]:
df1 = dict_df3[dict_df3['avg_tfidf']>threshold_tfidf]
df2 = dict_df3[dict_df3['score_chi']>threshold_chi]
df3 = dict_df3[dict_df3['score_llr']>threshold_llr]
df4 = dict_df3[dict_df3['score_emi']>threshold_emi]
df_vote = dict_df3
df_vote['vote']=0
print(df_vote)

      Unnamed: 0    id         term  avg_tfidf     score_chi  score_llr  \
0              0     1      abandon   0.005374  3.846051e+00   1.659552   
1              1     2          abc   0.015857  1.555876e+00   0.899990   
2              2     3  abcnews.com   0.033696  6.544514e-01   0.204845   
3              3     4     abdallah   0.006597  5.290393e-01   0.274067   
4              4     5        abdel   0.003924  7.944005e-01   0.411467   
5              5     6      abdomin   0.010949  5.290393e-01   0.274067   
6              6     7       abduct   0.016260  3.668540e-01   0.160386   
7              7     8        abdul   0.000851  9.267195e-01   0.464110   
8              8     9  abdul-karim   0.005135  2.937724e+00   1.519594   
9              9    10     abdullah   0.006808  1.045027e+01   5.383307   
10            10    11       abhorr   0.008349  4.723560e+00   0.599975   
11            11    12         abid   0.000591  8.999763e-01   0.378774   
12            12    13   

In [16]:
df_vote.loc[df1.id-1,'vote'] += 1
df_vote.loc[df2.id-1,'vote'] += 1
df_vote.loc[df3.id-1,'vote'] += 1
df_vote.loc[df4.id-1,'vote'] += 1
# df_vote

In [17]:
df_vote_ = df_vote[df_vote.vote>2] #(1,2)=>375 #(1,1)=>422 #(1.6,2)=>482 #(2,2)=>330 #(1,3)=>100
df_vote_ = df_vote_.filter(['id','term','vote'])
print(df_vote_)

        id         term  vote
12      13      abidjan     3
81      82         aden     3
130    131      african     3
134    135   aftershock     3
136    137       agassi     4
202    203      alassan     3
208    209      alberto     3
310    311         andr     3
385    386      appoint     3
396    397        april     3
473    474     ashcroft     3
520    521       attack     3
545    546   australian     3
625    626       ballot     3
651    652       barent     3
694    695         beat     3
721    722      belgrad     3
763    764         bhuj     3
857    858         boat     3
860    861         bodi     3
871    872         bomb     3
881    882         bone     3
955    956        bribe     3
1040  1041         buri     3
1057  1058  businessman     3
1101  1102     campaign     3
1108  1109       cancer     3
1109  1110       candid     3
1131  1132     capriati     4
1155  1156     carnahan     4
...    ...          ...   ...
7237  7238     slobodan     3
7392  7393

In [18]:
# df_vote_.to_csv('output/500terms_df_rev5.csv')

## Classifier
* 7-fold
* MNB
* BNB
* self-train
* ens voting (BNB lower weight)

In [19]:
df_vote = pd.read_csv('output/500terms_df_rev5.csv',index_col=False)
terms_li = list(set(df_vote.term.tolist()))

train_X = []
train_Y = []
print(len(terms_li))

186


In [20]:
with open('data/training.txt','r') as f:
    train_id = f.read().splitlines()
train_dict = {}

for trainid in train_id:
    trainid = trainid.split(' ')
    trainid = list(filter(None, trainid))
    train_dict[trainid[0]] = trainid[1:]
# train_dict #class:doc_id
train_dict = pickle.load(open('data/train_dict.pkl','rb'))
in_dir = 'data/IRTM/'
train_dict_ = {}
class_token = []
class_token_dict = {}
train_X = []
train_Y= []
train_ids = []
for c,d in tqdm(train_dict.items()):
    for doc in d:
        train_ids.append(doc)
        trainX = np.array([0]*len(terms_li))
        f = open('data/IRTM/'+doc+'.txt')
        texts = f.read()
        f.close()
        tokens_all = preprocess(texts)
        tokens_all = tokens_all.split(' ')
#         tokens_all = list(filter(None,tokens_all))
        tokens_all = dict(Counter(tokens_all))
        for key,value in tokens_all.items():
            if key in terms_li:
                trainX[terms_li.index(key)] = int(value)
#         trainX = np.array(trainX)
        
#         for token in tokens_all:
#             if token in terms_li:
#                 ind = terms_li.index(token)
#                 trainX[ind]+=1
        train_X.append(trainX)
        train_Y.append(int(c))
        
train_X = np.array(train_X)
train_Y = np.array(train_Y)

#         tokens_all = list(set(filter(None,tokens_all)))
#         class_token.append(tokens_all)
#     class_token_dict[c]=class_token
#     class_token=[]
# len(class_token_dict['1'])
# print(train_X.shape , train_Y.shape)

100%|██████████| 13/13 [00:07<00:00,  1.80it/s]


In [21]:
#建立term index matrix
tokens_all_class=[]
term_tf_mat=[]
for c,d in tqdm(train_dict.items()):
    for doc in d:
        f = open('data/IRTM/'+doc+'.txt')
        texts = f.read()
        f.close()
        tokens_all = preprocess(texts)
        tokens_all = tokens_all.split(' ')
        tokens_all = list(filter(None,tokens_all))
        tokens_all_class.extend(tokens_all)
    tokens_all = dict(Counter(tokens_all_class))
    term_tf_mat.append(tokens_all)

100%|██████████| 13/13 [00:07<00:00,  1.85it/s]


In [22]:
def train_MNB(train_set=train_dict,term_list=terms_li,term_tf_mat=term_tf_mat):
    prior = np.zeros(len(train_set))
    cond_prob = np.zeros((len(train_set), len(term_list)))
    
    for i,docs in train_set.items(): #13 classes 1~13
        prior[int(i)-1] = len(docs)/len(train_ids) #那個類別的文章有幾個/總共的文章數目 0~12
        token_count=0
        class_tf = np.zeros(len(term_list))
        for idx,term in enumerate(term_list):
            try:
                class_tf[idx] = term_tf_mat[int(i)-1][term]  #term在class的出現次數
            except:
                token_count+=1

        class_tf = class_tf + np.ones(len(term_list)) #smoothing (可改)
        class_tf = class_tf/(sum(class_tf) +token_count) #該class總共的token數(可改)
        cond_prob[int(i)-1] = class_tf #0~12
    return prior, cond_prob

In [23]:
prior,cond_prob = train_MNB()
print(prior,cond_prob)

[0.05496829 0.1141649  0.03488372 0.03276956 0.06976744 0.06553911
 0.06236786 0.05813953 0.27378436 0.04968288 0.06976744 0.06553911
 0.04862579] [[0.00048852 0.00048852 0.00048852 ... 0.00048852 0.00097704 0.00048852]
 [0.0376849  0.00105659 0.0001174  ... 0.0002348  0.01995774 0.00305236]
 [0.03390726 0.00095067 0.00010563 ... 0.00021126 0.01795711 0.00274638]
 ...
 [0.00917063 0.00505321 0.00232608 ... 0.00310144 0.01069462 0.00689803]
 [0.0083773  0.00461606 0.00212485 ... 0.00285756 0.01079523 0.00642341]
 [0.00802339 0.00444444 0.00203509 ... 0.00273684 0.01083041 0.00633918]]


In [24]:
def predict_MNB(test_id,prob=False,prior=prior,cond_prob=cond_prob,term_list=terms_li):
    f = open('data/IRTM/'+str(test_id)+'.txt')
    texts = f.read()
    f.close()
    tokens_all = preprocess(texts)
    tokens_all = tokens_all.split(' ')
    tokens_all = list(filter(None,tokens_all))
    
    class_scores = []
#     score = 0
    for i in range(13):
        score=0
#         print(prior[i])
        score += math.log(prior[i],10)
        for token in tokens_all:
            if token in term_list:
                score += math.log(cond_prob[i][term_list.index(token)])
        class_scores.append(score)
    if prob:
        return np.array(class_scores)
    else:
        return(np.argmax(class_scores)+1)

for testing class function only

In [25]:
def MNB(input_X,input_Y=None,prior_log_class=None,log_prob_feature=None,train=True,prob=False,smooth=1.0):
    if train:
        sample_num = input_X.shape[0]
        match_data = [[x for x, t in zip(input_X, input_Y) if t == c] for c in np.unique(input_Y)]
        prior_log_class = [np.log(len(i) / sample_num) for i in match_data]
        counts = np.array([np.array(i).sum(axis=0) for i in match_data]) + smooth
        log_prob_feature = np.log(counts / counts.sum(axis=1)[np.newaxis].T)
        return prior_log_class,log_prob_feature
    else:
        probability = [(log_prob_feature * x).sum(axis=1) + prior_log_class for x in input_X]
        if prob:
            return probability
        else:
            ans = np.argmax(probability,axis=1)
            return ans

In [26]:
class BernoulliNB(object):
    def __init__(self, alpha=1.0, binarize=0.0):
        self.alpha = alpha
        self.binarize = binarize
    def _binarize_X(self, X):
        return np.where(X > self.binarize, 1, 0) if self.binarize != None else X
    def fit(self, X, y):
        X = self._binarize_X(X)
        count_sample = X.shape[0]
        separated = [[x for x, t in zip(X, y) if t == c] for c in np.unique(y)]
        self.class_log_prior_ = [np.log(len(i) / count_sample) for i in separated]
        count = np.array([np.array(i).sum(axis=0) for i in separated]) + self.alpha
        smoothing = 2 * self.alpha
        n_doc = np.array([len(i) + smoothing for i in separated])
        self.feature_prob_ = count / n_doc[np.newaxis].T
        return self

    def predict_log_proba(self, X):
        X = self._binarize_X(X)
        return [(np.log(self.feature_prob_) * x + \
                 np.log(1 - self.feature_prob_) * np.abs(x - 1)
                ).sum(axis=1) + self.class_log_prior_ for x in X]

    def predict(self, X):
        X = self._binarize_X(X)
        return np.argmax(self.predict_log_proba(X), axis=1)

### Prediction

In [27]:
df_vote = pd.read_csv('output/500terms_df_rev5.csv',index_col=False)
terms_li = list(set(df_vote.term.tolist()))
print(len(terms_li))

186


In [28]:
with open('data/training.txt','r') as f:
    train_id = f.read().splitlines()
train_dict = {}
test_id = []
train_ids=[]
for trainid in train_id:
    trainid = trainid.split(' ')
    trainid = list(filter(None, trainid))
    train_ids.extend(trainid[1:])
for i in range(1095):
    if str(i+1) not in train_ids:
        test_id.append(i+1)
ans=[]
for doc in tqdm(test_id):
    ans.append(predict_MNB(doc))
print(ans)
df_ans = pd.DataFrame(list(zip(test_id,ans)),columns=['id','Value'])
# df_ans.to_csv('output/MNB.csv',index=False)
# df_ans

100%|██████████| 900/900 [00:11<00:00, 75.07it/s]

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 11, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 11, 2, 2, 4, 2, 2, 13, 9, 9, 9, 9, 11, 9, 9, 9, 11, 9, 9, 11, 9, 9, 9, 9, 10, 9, 9, 9, 13, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 2, 9, 9, 11, 9, 9, 11, 9, 9, 9, 9, 9, 9, 9, 13, 9, 13, 9, 9, 13, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 2, 9, 9, 9, 9, 9, 2, 9, 9, 9, 9, 9, 9, 9, 13, 9, 9, 9, 9, 9, 9, 9, 2, 9, 9, 9, 9, 9, 9, 9, 9, 9, 11, 9, 9, 11, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 1, 11, 9, 9, 11, 9, 9, 11, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 11, 9, 9, 12, 9, 9, 11, 11, 9, 9, 10, 11, 11, 11, 9, 9, 11, 11, 9, 11, 9, 11, 9, 1, 11, 9, 11, 9, 11, 11, 9, 10, 9, 9, 9, 1, 10, 10, 1, 9, 9, 11, 10, 1, 12, 9, 10, 1, 10, 1, 1, 9, 1, 9, 1, 1, 9, 10, 10, 10, 12, 10, 1, 12,

combine all prediction df

In [40]:
# import os
in_dir = './data/'
prefixed = [filename for filename in os.listdir(in_dir) if filename.startswith("MNB")]
df_from_each_file = [pd.read_csv(in_dir+f) for f in prefixed]
# prefixed

In [41]:
merged_df = functools.reduce(lambda left,right: pd.merge(left,right,on='id'), df_from_each_file)
merged_df.columns = ['id',0,1,2,3,4,5,6,7,8]
print(merged_df)

       id   0   1   2   3   4   5   6   7   8
0      17   2   2   2   2   2   2   2   2   2
1      18   2   2   2   2   2   2   2   2   2
2      20   2   2   2   2   2   2   2   2   2
3      21   2   2   2   2   2   2   2   2   2
4      22   2   2   2   2   2   2   9   2   2
5      23   2   2   2   2   2   2   2   2   2
6      24   2   2   2   2   2   2   2   2   2
7      25   2   2   2   2   2   2   2   2   2
8      26   9   2   2   2   2   2   9   2   2
9      27   2   2   2   2   2   2   2   2   2
10     28   2   2   2   2   2   2   9   2   2
11     30   2   2   2   2   2   2   2   2   2
12     32   2   2   2   2   2   2   9   2   2
13     33   2   2   2   2   2   2   2   2   2
14     34   2   2   2   2   2   2   2   2   2
15     35   2   2   2   2   2   2   2   2   2
16     36   2   2   2   2   2   2   9   2   2
17     37   2   2   2   2   2   2   9   2   2
18     38   2   2   2   2   2   2   2   2   2
19     39   2   2   2   2   2   2   2   2   2
20     40   2   2   2   2   2   2 

In [42]:
df01 = pd.DataFrame(merged_df.mode(axis=1)[0])
df02 = pd.DataFrame(merged_df['id'])
df_ans = pd.concat([df02,df01],axis=1)
df_ans = df_ans.astype('int')
df_ans.columns = ['id','Value']
df_ans.to_csv('output/voting_rev.csv',index=False)
print(df_ans)

       id  Value
0      17      2
1      18      2
2      20      2
3      21      2
4      22      2
5      23      2
6      24      2
7      25      2
8      26      2
9      27      2
10     28      2
11     30      2
12     32      2
13     33      2
14     34      2
15     35      2
16     36      2
17     37      2
18     38      2
19     39      2
20     40      2
21     41      2
22     42      2
23     43      2
24     45      2
25     46      2
26     47      2
27     48      2
28     49      2
29     50      2
..    ...    ...
870  1066      6
871  1067      9
872  1068      6
873  1069      6
874  1070      4
875  1071      6
876  1072      6
877  1073      4
878  1074      4
879  1075      6
880  1076     12
881  1077      6
882  1078      6
883  1079      9
884  1080      6
885  1081      6
886  1082      6
887  1083      6
888  1084      6
889  1085      6
890  1086      6
891  1087      6
892  1088      6
893  1089      6
894  1090      6
895  1091      6
896  1092     

do only in first time only ro produce new train_dict

In [32]:
# df1 = merged_df[(merged_df[0] == merged_df[1])&(merged_df[2]==merged_df[3])&(merged_df[1]==merged_df[2])]
# df1.reset_index(inplace=True,drop=True)
# df1['class'] = df1[0]
# df1 = df1.filter(['id','class'])
# df1

In [33]:
# df1[df1['class']=='1']

In [34]:
# with open('data/training.txt','r') as f:
#     train_id = f.read().splitlines()
# train_dict = {}
# for trainid in train_id:
#     trainid = trainid.split(' ')
#     trainid = list(filter(None, trainid))
#     train_dict[trainid[0]] = trainid[1:]
# train_dict #class:doc_id

In [35]:
# for i in range(13):
#     df1 = df1.astype(str)
#     li = df1[df1['class'] == str(i+1)]['id'].tolist()
#     train_dict[str(i+1)].extend(li)
# train_dict

In [36]:
# pickle.dump(obj=train_dict,file=open('data/train_dict.pkl','wb'))

In [37]:
with open('data/training.txt','r') as f:
    train_id = f.read().splitlines()
train_dict = {}
test_id = []
train_ids=[]
for trainid in train_id:
    trainid = trainid.split(' ')
    trainid = list(filter(None, trainid))
    train_ids.extend(trainid[1:])
for i in range(1095):
    if str(i+1) not in train_ids:
        test_id.append(i+1)
#     train_dict[trainid[0]] = trainid[1:]
# train_dict #class:doc_id
in_dir = 'data/IRTM/'
train_dict_ = {}
class_token = []
class_token_dict = {}
test_X = []
# train_Y= []
# for c,d in tqdm(train_dict.items()):
for doc in tqdm(test_id):
    testX = np.array([0]*len(terms_li))
    f = open('data/IRTM/'+str(doc)+'.txt')
    texts = f.read()
    f.close()
    tokens_all = preprocess(texts)
    tokens_all = tokens_all.split(' ')
#         tokens_all = list(filter(None,tokens_all))
    tokens_all = dict(Counter(tokens_all))
    for key,value in tokens_all.items():
        if key in terms_li:
            testX[terms_li.index(key)] = int(value)

    test_X.append(testX)
        
test_X = np.array(test_X)
# print(test_X.shape)

100%|██████████| 900/900 [00:06<00:00, 131.85it/s]


In [38]:
# print(train_X.shape, train_Y.shape , test_X.shape)

In [39]:
# df_ans = pd.DataFrame(list(zip(test_id,ans2)),columns=['id','Value'])
# df_ans.to_csv('output/MNB02.csv',index=False)
# df_ans